# Handling double gameweeks

In this notebook we revisit the fixture data and account for double gameweeks. The FFS data collapses double gameweek fixtures into a single row however our fixture data has 2 rows for each fixture. We make sure that these sources are consistent with each other and generate any 'next fixture' features.

## Set-up

In [1]:
import getpass
from functools import reduce
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from s3fs import S3FileSystem
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
pd.options.display.max_columns = None

In [3]:
os.chdir('../..')

In [4]:
AWS_ACCESS_KEY_ID = getpass.getpass()
AWS_SECRET_KEY = getpass.getpass()

 ····················
 ········································


In [5]:
s3_filesystem = S3FileSystem(
    key=AWS_ACCESS_KEY_ID,
    secret=AWS_SECRET_KEY
)

## Load FFS data

In [6]:
ffs_all_data_1718 = pq.read_table(
    f"s3://fantasy-football-scout/processed/fantasy_football_scout_all_features_1718_to_1819.parquet", 
    filesystem=s3_filesystem
).to_pandas()

print(ffs_all_data_1718.shape)
ffs_all_data_1718.head()

(20449, 308)


,Name,Team,Aerial Duels - Won - Percentage,Aerial Duels Lost,Aerial Duels Total,Aerial Duels Won,Appearances,Assists,Attempts From Crossed Corners,Attempts From Direct Free-kick,Attempts From Indirect Free-kick,Attempts From Set Plays,position,full_name,season,gw,Attempts From Set Plays Conceded,Attempts from Throw-in,BPS Baseline,BPS CBI,BPS Clean Sheets,BPS Goals,BPS Pass Completion,BPS Recoveries,BPS Time Played,BPS Yellow Cards,Bad Touches,Big Chances Conceded,Big Chances Created,Big Chances Missed,Big Chances Missed Ball,Big Chances Missed Shots,Big Chances Scored,Big Chances Total,Blocks,Bonus Points,CBI,Caught Offside,Chances Conceded (Centre Zone),Chances Conceded (Left Zone),Chances Conceded (Right Zone),Chances Created,Chances Created (Centre Zone),Chances Created (Left Zone),Chances Created (Percentage Centre Zone),Chances Created (Percentage Left Zone),Chances Created (Percentage Right Zone),Chances Created (Right Zone),Chances From Counter Attack,Chances From Open Play,Chances Missed,Clean Sheets,Clearances,Clearances - Successful,Clearances - Unsuccessful,Corner Conversion,Corners,Corners - Successful,Corners - Unsuccessful,Cross Completion,Crosses,Crosses - Open Play - Successful,Crosses - Open Play - Unsuccessful,Crosses - Opponents Half (Left Zone),Crosses - Opponents Half (Percentage Left Zone),Crosses - Opponents Half (Percentage Right Zone),Crosses - Opponents Half (Right Zone),Crosses - Successful,Crosses - Unsuccessful,Crosses Conceded (Left Zone),Crosses Conceded (Right Zone),Crosses From Open Play,Dispossessed,Distribution - Kicks From Hands,Distribution - Successful,Distribution - Throws,Distribution - Unsuccessful,Dribbles,Dribbles - Failed,Dribbles - Successful,Dribbles - Successful Percentage,Duels,Duels Lost,Duels Won,Errors Leading to Chance,Errors Leading to Goal,Exact Time Played,FPL Goal Involvement,Fantasy Assists,Flick Ons,Fouls,Fouls Conceded,Fouls Won,Games Played,Goal Attempts,Goal Attempts Conceded,Goal Attempts In Box Conceded,Goal Attempts Outside Box Conceded,Goal Conversion,Goal Involvement,Goal Kicks,Goals,Goals Conceded,Goals From Counter Attack,Goals From Direct Corners,Goals From Direct Free-kick,Goals From Indirect Free-kick,Goals From Inside Box,Goals From Open Play,Goals From Outside Box,Goals From Penalties,Goals From Set Plays,Goals From Throw-in,Ground Duels,Ground Duels Lost,Ground Duels Won,Handballs,Headed Attempts From Set Plays,Headed Clearances,Headed Clearances - Successful,Headed Goal Attempts,Headed Goal Attempts - Off Target,Headed Goal Attempts - On Target,Headed Goal Attempts Conceded,Headed Goals,Headers Off Target,Headers On Goal,Hit Woodwork,ICT Creativity,ICT Index,ICT Influence,ICT Threat,Interceptions,Key Contributions,Key Passes,Launches,Launches - Successful,Launches - Unsuccessful,Lay-offs,Long Clearances,Long Passes,Minutes Per Baseline BPS,Minutes Per Big Chance,Minutes Per Big Chance Conceded,Minutes Per Block,Minutes Per Chance,Minutes Per Chance Conceded,Minutes Per Chances Created,Minutes Per Clearance,Minutes Per Cross,Minutes Per Goal,Minutes Per Goal Attempt In Box,Minutes Per Interception,Minutes Per Pass Received,Minutes Per Recovery,Minutes Per Save,Minutes Per Shot On Target,Minutes Per Tackle,Minutes Per Tackle Won,Minutes Per Touch,Minutes Per xA,Minutes Per xG,Minutes Per xG Conceded,Minutes Per xG On Target Conceded,Minutes Per xGI,Minutes Per xGP,Missed Opportunities,Net Tackles,Open Play Goal Attempts,Open Play Goal Attempts Conceded,Overall Points,Own Goals,Ownership,Pass Completion,Pass Completion - Final Third,Pass Completion - Opponents Half,Passes,Passes - Backward,Passes - Final Third,Passes - Final Third - Successful,Passes - Final Third - Unsuccessful,Passes - Forward,Passes - Forward - Successful,Passes - Forward - Unsuccessful,Passes - Long - Successful,Passes - Long - Unsuccessful,Passes - Opponents Half,Passes - Opponents Half (Centre),Passes - Opponents Half (Left),Passes - Opponents Half (Right),Passes - Opponents Half - 

## Load fixture and odds data

In [7]:
fixture_and_odds_2016_to_2019 = pd.read_parquet('data/processed/fixture_and_odds_2016_to_2019.parquet')
print(fixture_and_odds_2016_to_2019.shape)
fixture_and_odds_2016_to_2019.head()

(1140, 7)


,season,gw,home_team,away_team,home_win,draw,away_win
0,2016-17,1,Chelsea,West Ham United,0.52,3.32,5.98
1,2016-17,2,West Ham United,Bournemouth,1.41,2.29,2.20
2,2016-17,3,Manchester City,West Ham United,0.27,5.42,10.92
3,2016-17,4,West Ham United,Watford,0.88,2.58,3.49
4,2016-17,5,West Bromwich Albion,West Ham United,1.82,2.21,1.74


Find example of a double gameweek

In [8]:
fixture_and_odds_2016_to_2019[
    (fixture_and_odds_2016_to_2019['season'] == '2017-18') & 
    (fixture_and_odds_2016_to_2019['gw'] == 37)
]

,season,gw,home_team,away_team,home_win,draw,away_win
415,2017-18,37,Leicester City,West Ham United,1.28,2.49,2.31
416,2017-18,37,West Ham United,Manchester United,3.57,2.85,0.81
452,2017-18,37,Arsenal,Burnley,0.43,3.93,6.86
486,2017-18,37,Manchester City,Huddersfield Town,0.09,12.16,27.54
487,2017-18,37,Chelsea,Huddersfield Town,0.26,4.84,13.75
520,2017-18,37,Leicester City,Arsenal,2.86,2.87,0.96
549,2017-18,37,Watford,Newcastle United,1.19,2.31,2.69
577,2017-18,37,Bournemouth,Swansea City,1.01,2.60,2.95
603,2017-18,37,Chelsea,Liverpool,1.20,2.68,2.34
612,2017-18,37,Everton,Southampton,1.80,2.30,1.70


Find player from double gameweek in FFS data

In [9]:
ffs_all_data_1718[
    (ffs_all_data_1718['Team'] == 'LEI') & 
    (ffs_all_data_1718['season'] == '2017-2018') & 
    (ffs_all_data_1718['gw'] == 37)
]['Time Played']

1290     180
1313     117
1318     133
3812      90
3814      90
7387      82
7409     180
7422     180
7448     180
7455      53
7470      90
15047    180
15069    180
15088     45
15099     47
15117    151
15127      2
Name: Time Played, dtype: int64

FFS minutes >90 as performance in both matches is combined

## Reformat fixture data

In [10]:
fixture_and_odds_2016_to_2019.head()

,season,gw,home_team,away_team,home_win,draw,away_win
0,2016-17,1,Chelsea,West Ham United,0.52,3.32,5.98
1,2016-17,2,West Ham United,Bournemouth,1.41,2.29,2.20
2,2016-17,3,Manchester City,West Ham United,0.27,5.42,10.92
3,2016-17,4,West Ham United,Watford,0.88,2.58,3.49
4,2016-17,5,West Bromwich Albion,West Ham United,1.82,2.21,1.74


In [11]:
# Amend fixture data so that it is at a team level and specifies whether that team played home or away. This doubles the size of the dataset.
teams_in_gw_1 = fixture_and_odds_2016_to_2019.copy()
teams_in_gw_1['team_name'] = teams_in_gw_1['home_team']
teams_in_gw_1['team_name_opponent'] = teams_in_gw_1['away_team']
teams_in_gw_1['was_home'] = True

teams_in_gw_2 = fixture_and_odds_2016_to_2019.copy()
teams_in_gw_2['team_name'] = teams_in_gw_2['away_team']
teams_in_gw_2['team_name_opponent'] = teams_in_gw_2['home_team']
teams_in_gw_2['was_home'] = False

teams_in_gw = teams_in_gw_1.append(teams_in_gw_2)
teams_in_gw.head()

,season,gw,home_team,away_team,home_win,draw,away_win,team_name,team_name_opponent,was_home
0,2016-17,1,Chelsea,West Ham United,0.52,3.32,5.98,Chelsea,West Ham United,True
1,2016-17,2,West Ham United,Bournemouth,1.41,2.29,2.20,West Ham United,Bournemouth,True
2,2016-17,3,Manchester City,West Ham United,0.27,5.42,10.92,Manchester City,West Ham United,True
3,2016-17,4,West Ham United,Watford,0.88,2.58,3.49,West Ham United,Watford,True
4,2016-17,5,West Bromwich Albion,West Ham United,1.82,2.21,1.74,West Bromwich Albion,West Ham United,True


In [12]:
teams_in_gw.shape

(2280, 10)

In [13]:
teams_in_gw['win_odds'] = np.where(
    teams_in_gw['was_home'],
    teams_in_gw['home_win'],
    teams_in_gw['away_win']
)

teams_in_gw['lose_odds'] = np.where(
    teams_in_gw['was_home'],
    teams_in_gw['away_win'],
    teams_in_gw['home_win']
)

In [14]:
teams_in_gw.drop(columns=['home_team', 'away_team', 'home_win', 'away_win'], inplace=True)

In [15]:
teams_in_gw.head()

,season,gw,draw,team_name,team_name_opponent,was_home,win_odds,lose_odds
0,2016-17,1,3.32,Chelsea,West Ham United,True,0.52,5.98
1,2016-17,2,2.29,West Ham United,Bournemouth,True,1.41,2.20
2,2016-17,3,5.42,Manchester City,West Ham United,True,0.27,10.92
3,2016-17,4,2.58,West Ham United,Watford,True,0.88,3.49
4,2016-17,5,2.21,West Bromwich Albion,West Ham United,True,1.82,1.74


## Add further team details

In [16]:
team_data = pd.read_csv('data/external/team_season_data.csv')
print(team_data.shape)
team_data.head()

(80, 5)


,team,team_name,promoted_side,top_6_last_season,season
0,1,Arsenal,0,1,2016-17
1,2,Bournemouth,0,0,2016-17
2,3,Burnley,1,0,2016-17
3,4,Chelsea,0,0,2016-17
4,5,Crystal Palace,0,0,2016-17


In [17]:
team_data_opponent = team_data.copy()
team_data_opponent.drop('team', axis=1, inplace=True)
team_data_opponent.columns = [col + '_opponent' for col in team_data_opponent.columns]
team_data_opponent.rename(columns={'season_opponent': 'season'}, inplace=True)
team_data_opponent.head()

,team_name_opponent,promoted_side_opponent,top_6_last_season_opponent,season
0,Arsenal,0,1,2016-17
1,Bournemouth,0,0,2016-17
2,Burnley,1,0,2016-17
3,Chelsea,0,0,2016-17
4,Crystal Palace,0,0,2016-17


In [18]:
teams_in_gw.head()

,season,gw,draw,team_name,team_name_opponent,was_home,win_odds,lose_odds
0,2016-17,1,3.32,Chelsea,West Ham United,True,0.52,5.98
1,2016-17,2,2.29,West Ham United,Bournemouth,True,1.41,2.20
2,2016-17,3,5.42,Manchester City,West Ham United,True,0.27,10.92
3,2016-17,4,2.58,West Ham United,Watford,True,0.88,3.49
4,2016-17,5,2.21,West Bromwich Albion,West Ham United,True,1.82,1.74


In [19]:
teams_in_gw = teams_in_gw.merge(
    team_data_opponent,
    on=['team_name_opponent', 'season'],
    how='left'
)

In [20]:
teams_in_gw = teams_in_gw.merge(
    team_data.drop('team', axis=1),
    on=['team_name', 'season'],
    how='left'
)

In [21]:
teams_in_gw.head()

,season,gw,draw,team_name,team_name_opponent,was_home,win_odds,lose_odds,promoted_side_opponent,top_6_last_season_opponent,promoted_side,top_6_last_season
0,2016-17,1,3.32,Chelsea,West Ham United,True,0.52,5.98,0,0,0,0
1,2016-17,2,2.29,West Ham United,Bournemouth,True,1.41,2.20,0,0,0,0
2,2016-17,3,5.42,Manchester City,West Ham United,True,0.27,10.92,0,0,0,1
3,2016-17,4,2.58,West Ham United,Watford,True,0.88,3.49,0,0,0,0
4,2016-17,5,2.21,West Bromwich Albion,West Ham United,True,1.82,1.74,0,0,0,0


In [22]:
teams_in_gw.rename(columns={'draw': 'draw_odds'}, inplace=True)

In [23]:
teams_in_gw.head()

,season,gw,draw_odds,team_name,team_name_opponent,was_home,win_odds,lose_odds,promoted_side_opponent,top_6_last_season_opponent,promoted_side,top_6_last_season
0,2016-17,1,3.32,Chelsea,West Ham United,True,0.52,5.98,0,0,0,0
1,2016-17,2,2.29,West Ham United,Bournemouth,True,1.41,2.20,0,0,0,0
2,2016-17,3,5.42,Manchester City,West Ham United,True,0.27,10.92,0,0,0,1
3,2016-17,4,2.58,West Ham United,Watford,True,0.88,3.49,0,0,0,0
4,2016-17,5,2.21,West Bromwich Albion,West Ham United,True,1.82,1.74,0,0,0,0


In [25]:
# Double gameweeks
teams_in_gw[teams_in_gw['season'] == '2017-18'].groupby(['gw', 'team_name']).count()['season'][
    teams_in_gw[teams_in_gw['season'] == '2017-18'].groupby(['gw', 'team_name']).count()['season'] == 2
]

gw  team_name             
22  Tottenham Hotspur         2
    West Ham United           2
34  Bournemouth               2
    Brighton & Hove Albion    2
    Burnley                   2
    Chelsea                   2
    Leicester City            2
    Manchester United         2
    Southampton               2
    Tottenham Hotspur         2
37  Arsenal                   2
    Brighton & Hove Albion    2
    Chelsea                   2
    Huddersfield Town         2
    Leicester City            2
    Manchester City           2
    Manchester United         2
    Newcastle United          2
    Southampton               2
    Swansea City              2
    Tottenham Hotspur         2
    West Ham United           2
Name: season, dtype: int64

In [26]:
teams_in_gw[
    (teams_in_gw['season'] == '2017-18') & (teams_in_gw['gw'] == 37) & (teams_in_gw['team_name'] == 'Arsenal')
]

,season,gw,draw_odds,team_name,team_name_opponent,was_home,win_odds,lose_odds,promoted_side_opponent,top_6_last_season_opponent,promoted_side,top_6_last_season
452,2017-18,37,3.93,Arsenal,Burnley,True,0.43,6.86,0,0,0,1
1660,2017-18,37,2.87,Arsenal,Leicester City,False,0.96,2.86,0,0,0,1


We want to collapse these cases into a single row. We will do the following aggregations:

- `draw_odds`: mean
- `win_odds`: mean
- `lose_odds`: mean
- `was_home`: sum (rename to number of home matches)
- `promoted_side_opponent`: sum (number of)
- `top_6_last_season_opponent`: sum (number of)
- `promoted_side`: mean (get original value back)
- `top_6_last_season`: mean (get original value back)
- Also add `number_of_matches` to identify double gameweeks

In [27]:
teams_in_gw.rename(
    columns={
        'was_home': 'number_of_home_matches',
        'promoted_side_opponent': 'number_of_promoted_side_opponent',
        'top_6_last_season_opponent': 'number_of_top_6_last_season_opponent',
    },
    inplace=True
)

In [28]:
teams_in_gw['number_of_matches'] = 1

In [29]:
teams_in_gw.head()

,season,gw,draw_odds,team_name,team_name_opponent,number_of_home_matches,win_odds,lose_odds,number_of_promoted_side_opponent,number_of_top_6_last_season_opponent,promoted_side,top_6_last_season,number_of_matches
0,2016-17,1,3.32,Chelsea,West Ham United,True,0.52,5.98,0,0,0,0,1
1,2016-17,2,2.29,West Ham United,Bournemouth,True,1.41,2.20,0,0,0,0,1
2,2016-17,3,5.42,Manchester City,West Ham United,True,0.27,10.92,0,0,0,1,1
3,2016-17,4,2.58,West Ham United,Watford,True,0.88,3.49,0,0,0,0,1
4,2016-17,5,2.21,West Bromwich Albion,West Ham United,True,1.82,1.74,0,0,0,0,1


In [30]:
agg_dict = {
    'draw_odds': 'mean',
    'win_odds': 'mean',
    'lose_odds': 'mean',
    'number_of_home_matches': 'sum',
    'number_of_promoted_side_opponent': 'sum',
    'number_of_top_6_last_season_opponent': 'sum',
    'promoted_side': 'mean',
    'top_6_last_season': 'mean',
    'number_of_matches': 'sum'
}

teams_in_gw_agg = teams_in_gw.groupby(['season', 'gw', 'team_name']).agg(agg_dict).reset_index()

Check Arsenal double gameweek from above

In [31]:
teams_in_gw_agg.head()

,season,gw,team_name,draw_odds,win_odds,lose_odds,number_of_home_matches,number_of_promoted_side_opponent,number_of_top_6_last_season_opponent,promoted_side,top_6_last_season,number_of_matches
0,2016-17,1,Arsenal,2.34,1.73,1.72,1.0,0,0,0,1,1
1,2016-17,1,Bournemouth,2.59,4.40,0.76,1.0,0,1,0,0,1
2,2016-17,1,Burnley,2.16,1.74,1.84,1.0,0,0,1,0,1
3,2016-17,1,Chelsea,3.32,0.52,5.98,1.0,0,0,0,0,1
4,2016-17,1,Crystal Palace,2.17,1.17,2.90,1.0,0,0,0,0,1


In [32]:
teams_in_gw_agg[
    (teams_in_gw_agg['season'] == '2017-18') & (teams_in_gw_agg['gw'] == 37) & (teams_in_gw_agg['team_name'] == 'Arsenal')
]

,season,gw,team_name,draw_odds,win_odds,lose_odds,number_of_home_matches,number_of_promoted_side_opponent,number_of_top_6_last_season_opponent,promoted_side,top_6_last_season,number_of_matches
1441,2017-18,37,Arsenal,3.4,0.695,4.86,1.0,0,0,0,1,2


Now a single row!

In [33]:
teams_in_gw_agg.shape

(2219, 12)

In [35]:
# Double gameweeks with both fixtures at home
teams_in_gw_agg[teams_in_gw_agg['number_of_home_matches'] == teams_in_gw_agg['number_of_home_matches'].max()]

,season,gw,team_name,draw_odds,win_odds,lose_odds,number_of_home_matches,number_of_promoted_side_opponent,number_of_top_6_last_season_opponent,promoted_side,top_6_last_season,number_of_matches
712,2016-17,37,Manchester City,7.135,0.185,14.840,2.0,0,1,0,1,2
1392,2017-18,34,Burnley,2.370,2.560,1.460,2.0,0,1,0,0,2
1445,2017-18,37,Chelsea,3.760,0.730,8.045,2.0,1,1,0,1,2
1449,2017-18,37,Leicester City,2.680,2.070,1.635,2.0,0,1,0,0,2
1451,2017-18,37,Manchester City,9.330,0.130,21.305,2.0,2,0,0,1,2
1968,2018-19,25,Everton,3.655,5.340,1.080,2.0,1,1,0,0,2
2121,2018-19,34,Brighton & Hove Albion,2.265,1.155,2.910,2.0,1,0,0,0,2
2158,2018-19,35,Wolverhampton Wanderers,2.285,1.330,2.825,2.0,0,1,1,0,2


In [36]:
teams_in_gw[
    (teams_in_gw['season'] == '2018-19') & (teams_in_gw['gw'] == 34) & (teams_in_gw['team_name'] == 'Brighton & Hove Albion')
]

,season,gw,draw_odds,team_name,team_name_opponent,number_of_home_matches,win_odds,lose_odds,number_of_promoted_side_opponent,number_of_top_6_last_season_opponent,promoted_side,top_6_last_season,number_of_matches
1047,2018-19,34,2.27,Brighton & Hove Albion,Bournemouth,True,1.31,2.39,0,0,0,0,1
1119,2018-19,34,2.26,Brighton & Hove Albion,Cardiff City,True,1.00,3.43,1,0,0,0,1


## Double gameweek flag

In [37]:
teams_in_gw_agg.head()

,season,gw,team_name,draw_odds,win_odds,lose_odds,number_of_home_matches,number_of_promoted_side_opponent,number_of_top_6_last_season_opponent,promoted_side,top_6_last_season,number_of_matches
0,2016-17,1,Arsenal,2.34,1.73,1.72,1.0,0,0,0,1,1
1,2016-17,1,Bournemouth,2.59,4.40,0.76,1.0,0,1,0,0,1
2,2016-17,1,Burnley,2.16,1.74,1.84,1.0,0,0,1,0,1
3,2016-17,1,Chelsea,3.32,0.52,5.98,1.0,0,0,0,0,1
4,2016-17,1,Crystal Palace,2.17,1.17,2.90,1.0,0,0,0,0,1


In [38]:
teams_in_gw_agg['number_of_matches'].value_counts()

1    2158
2      61
Name: number_of_matches, dtype: int64

In [39]:
teams_in_gw_agg['double_gameweek'] = np.where(
    teams_in_gw_agg['number_of_matches'] == 2,
    1,
    0
)

In [40]:
teams_in_gw_agg[
    (teams_in_gw_agg['season'] == '2017-18') & (teams_in_gw_agg['gw'] == 37) & (teams_in_gw_agg['team_name'] == 'Arsenal')
]

,season,gw,team_name,draw_odds,win_odds,lose_odds,number_of_home_matches,number_of_promoted_side_opponent,number_of_top_6_last_season_opponent,promoted_side,top_6_last_season,number_of_matches,double_gameweek
1441,2017-18,37,Arsenal,3.4,0.695,4.86,1.0,0,0,0,1,2,1


In [41]:
teams_in_gw_agg.drop('number_of_matches', axis=1, inplace=True)

## Next match/gameweek features

In [42]:
teams_in_gw_agg.sort_values(['season', 'team_name', 'gw'], inplace=True)
teams_in_gw_agg.head()

,season,gw,team_name,draw_odds,win_odds,lose_odds,number_of_home_matches,number_of_promoted_side_opponent,number_of_top_6_last_season_opponent,promoted_side,top_6_last_season,double_gameweek
0,2016-17,1,Arsenal,2.34,1.73,1.72,1.0,0,0,0,1,0
20,2016-17,2,Arsenal,2.51,1.58,1.79,0.0,0,1,0,1,0
40,2016-17,3,Arsenal,3.00,0.64,4.82,0.0,0,0,0,1,0
60,2016-17,4,Arsenal,3.00,0.61,5.18,1.0,0,1,0,1,0
80,2016-17,5,Arsenal,3.08,0.62,4.99,0.0,1,0,0,1,0


Group by `team_name` only (not `season` as well) so that gameweek 1 predictions rely on previous season data. GW38 of latest season will have nulls but we can drop these.

In [43]:
for feature in [
    'draw_odds', 
    'win_odds', 
    'lose_odds', 
    'number_of_home_matches', 
    'number_of_promoted_side_opponent', 
    'number_of_top_6_last_season_opponent',
    'double_gameweek'
]:
    teams_in_gw_agg[f'next_gameweek_{feature}'] = teams_in_gw_agg.groupby(['team_name'])[feature].shift(-1)

In [44]:
teams_in_gw_agg.head()

,season,gw,team_name,draw_odds,win_odds,lose_odds,number_of_home_matches,number_of_promoted_side_opponent,number_of_top_6_last_season_opponent,promoted_side,top_6_last_season,double_gameweek,next_gameweek_draw_odds,next_gameweek_win_odds,next_gameweek_lose_odds,next_gameweek_number_of_home_matches,next_gameweek_number_of_promoted_side_opponent,next_gameweek_number_of_top_6_last_season_opponent,next_gameweek_double_gameweek
0,2016-17,1,Arsenal,2.34,1.73,1.72,1.0,0,0,0,1,0,2.51,1.58,1.79,0.0,0.0,1.0,0.0
20,2016-17,2,Arsenal,2.51,1.58,1.79,0.0,0,1,0,1,0,3.00,0.64,4.82,0.0,0.0,0.0,0.0
40,2016-17,3,Arsenal,3.00,0.64,4.82,0.0,0,0,0,1,0,3.00,0.61,5.18,1.0,0.0,1.0,0.0
60,2016-17,4,Arsenal,3.00,0.61,5.18,1.0,0,1,0,1,0,3.08,0.62,4.99,0.0,1.0,0.0,0.0
80,2016-17,5,Arsenal,3.08,0.62,4.99,0.0,1,0,0,1,0,2.43,1.42,2.07,1.0,0.0,0.0,0.0


Check `next_gameweek_double_gameweek` for Arsenal example

In [45]:
teams_in_gw_agg[
    (teams_in_gw_agg['season'] == '2017-18') & (teams_in_gw_agg['gw'] == 36) & (teams_in_gw_agg['team_name'] == 'Arsenal')
]

,season,gw,team_name,draw_odds,win_odds,lose_odds,number_of_home_matches,number_of_promoted_side_opponent,number_of_top_6_last_season_opponent,promoted_side,top_6_last_season,double_gameweek,next_gameweek_draw_odds,next_gameweek_win_odds,next_gameweek_lose_odds,next_gameweek_number_of_home_matches,next_gameweek_number_of_promoted_side_opponent,next_gameweek_number_of_top_6_last_season_opponent,next_gameweek_double_gameweek
1421,2017-18,36,Arsenal,4.35,7.18,0.39,0.0,0,1,0,1,0,3.4,0.695,4.86,1.0,0.0,0.0,1.0


In [46]:
teams_in_gw_agg[
    (teams_in_gw_agg['season'] == '2018-19') & (teams_in_gw_agg['gw'] == 38) & (teams_in_gw_agg['team_name'] == 'Arsenal')
]

,season,gw,team_name,draw_odds,win_odds,lose_odds,number_of_home_matches,number_of_promoted_side_opponent,number_of_top_6_last_season_opponent,promoted_side,top_6_last_season,double_gameweek,next_gameweek_draw_odds,next_gameweek_win_odds,next_gameweek_lose_odds,next_gameweek_number_of_home_matches,next_gameweek_number_of_promoted_side_opponent,next_gameweek_number_of_top_6_last_season_opponent,next_gameweek_double_gameweek
2199,2018-19,38,Arsenal,2.65,1.75,1.54,0.0,0,0,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
teams_in_gw_agg.isnull().sum()

season                                                 0
gw                                                     0
team_name                                              0
draw_odds                                              0
win_odds                                               0
lose_odds                                              0
number_of_home_matches                                 0
number_of_promoted_side_opponent                       0
number_of_top_6_last_season_opponent                   0
promoted_side                                          0
top_6_last_season                                      0
double_gameweek                                        0
next_gameweek_draw_odds                               26
next_gameweek_win_odds                                26
next_gameweek_lose_odds                               26
next_gameweek_number_of_home_matches                  26
next_gameweek_number_of_promoted_side_opponent        26
next_gameweek_number_of_top_6_l

In [48]:
teams_in_gw_agg.shape

(2219, 19)

In [49]:
teams_in_gw_agg.dropna(axis=0, inplace=True)

In [50]:
teams_in_gw_agg.shape

(2193, 19)

## Save

Given we are only doing feature selection at this stage we will not formalise these steps into (tested) functions. This is because many of these features may turn out to be unimportant.

After the final feature set is determined this notebook can be used to find the relevant code chunks for processing.

For now save as a parquet.

In [52]:
teams_in_gw_agg.to_parquet('data/processed/formatted_fixture_and_odds_features_2016_to_2019.parquet', index=False)